<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FP_AGENT_CREWAI_DEEPSEEK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cell 1: Install Necessary Libraries

In [ ]:
!pip install colab-env -q
import colab_env

In [ ]:
!pip install crewai -q
!pip install 'crewai[tools]' -q
!pip install openai -q

In [4]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from openai import OpenAI
from pydantic import Field

## FP Agents

In [9]:
# --- Cell 2: Set Up DeepSeek API Key ---
try:
    from google.colab import userdata
    deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
except ImportError:
    print("google.colab not found. Assuming DEEPSEEK_API_KEY is set as an environment variable.")
    deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if not deepseek_api_key:
    raise ValueError("DEEPSEEK_API_KEY not found. Please set it in Colab secrets or as an environment variable.")

# --- Define Custom Tools for Flight Planning ---

class AviationDatabaseSearchTool(BaseTool):
    name: str = "AviationDatabaseSearchTool"
    description: str = "A tool for searching aviation databases for NOTAMs, airport information, airspace restrictions, and weather data."

    def _run(self, query: str) -> str:
        # Simulated responses for demonstration
        if "NOTAMs" in query:
            return f"Aviation database search results for '{query}': NOTAMs indicate temporary flight restriction near XYZ airport from 1400Z-1800Z due to airshow. (Simulated)"
        elif "airport information" in query:
            return f"Aviation database search results for '{query}': KORD: Runways 09L/27R, 09R/27L active. EDDF: Runways 07L/25R, 07R/25L active. Fuel available at both. (Simulated)"
        elif "weather" in query:
            # Simulate a relevant weather report for the NAT selection
            return f"Aviation database search results for '{query}': KORD METAR: 062200Z 27010KT 10SM BKN050 18/08 A2995. EDDF METAR: 070400Z 23005KT 9999 FEW020 15/10 Q1012. Significant en-route weather: Moderate turbulence over Greenland, strong jet stream at FL350-FL390 with winds up to 150 kts, strongest favoring a more northerly track. (Simulated)"
        elif "maintenance records" in query:
            return f"Aviation database search results for '{query}': Boeing 777-300ER (Tail N777UA) last major inspection (C-check) completed on 2025-05-15. No deferred defects relevant to flight payload. (Simulated)"
        elif "passenger manifest" in query:
            return f"Aviation database search results for '{query}': Estimated 250 adult passengers and 20 children for KORD-EDDF flight. Average adult weight 190 lbs, average child weight 80 lbs. Estimated 15,000 lbs of baggage. (Simulated)"
        else:
            return f"Aviation database search results for '{query}': No specific details found, general aviation data available. (Simulated)"

class AirTrafficControlTool(BaseTool):
    name: str = "AirTrafficControlTool"
    description: str = "A tool for querying current air traffic control instructions, clearances, and real-time traffic data, including dynamic North Atlantic Tracks (NATs) and their waypoints."

    def _run(self, query: str) -> str:
        # Simulated dynamic NATs for demonstration, based on a query for specific day/direction
        if "NATs for eastbound traffic June 7 2025" in query:
            # In a real scenario, this would call an API or parse a daily NOTAM/track message (e.g., from an OFP provider)
            nat_data = {
                "NAT A": "ENTRY: N50W020 EXIT: N52W050 (Waypoints: 50N/020W, 50N/030W, 51N/040W, 52N/050W) - More southerly, might face headwind.",
                "NAT B": "ENTRY: N49W015 EXIT: N51N055 (Waypoints: 49N/015W, 49N/025W, 50N/035W, 51N/045W, 51N/055W) - Balanced track.",
                "NAT C": "ENTRY: N48W010 EXIT: N50W060 (Waypoints: 48N/010W, 48N/020W, 49N/030W, 49N/040W, 50N/050W, 50N/060W) - More northerly, currently offers best tailwinds.",
                "Recommended Eastbound (considering simulated winds)": "NAT C - Offering most favorable winds and efficient routing for KORD-EDDF."
            }
            return f"Air Traffic Control data for '{query}': {nat_data} (Simulated based on typical NAT structure and wind preference)"
        elif "oceanic clearance" in query:
            return f"Air Traffic Control data for '{query}': Current clearances suggest a routing via oceanic entry point DOGAL, altitude FL370. Expect oceanic clearance 30 mins prior to entry. (Simulated)"
        return f"Air Traffic Control data for '{query}': No specific ATC details found for query. (Simulated)"

class DeepSeekFlightAI(BaseTool):
    name: str = "DeepSeekFlightAI"
    description: str = "A powerful AI model for generating responses based on aviation prompts and complex reasoning, specifically for flight planning."

    client: OpenAI = Field(default=None, exclude=True)
    model_name: str = Field(default='deepseek-reasoner') # Default to reasoner for complex flight planning tasks

    def __init__(self, api_key: str, model_name: str = 'deepseek-reasoner', **data):
        super().__init__(**data)
        self.client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
        self.model_name = model_name

    def _run(self, prompt: str) -> str:
        if not isinstance(prompt, str):
            if isinstance(prompt, dict) and 'description' in prompt:
                actual_prompt = prompt['description']
            else:
                raise ValueError(f"Expected prompt to be a string or a dictionary with 'description', but received {type(prompt)}")
        else:
            actual_prompt = prompt

        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": "You are a helpful AI assistant specializing in flight planning and aviation."},
                    {"role": "user", "content": actual_prompt},
                ],
                stream=False
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error interacting with DeepSeek API for flight planning: {e}"

# Instantiate flight planning tools
aviation_database_search_tool = AviationDatabaseSearchTool()
air_traffic_control_tool = AirTrafficControlTool()

deepseek_reasoner_flight_ai = DeepSeekFlightAI(
    api_key=deepseek_api_key,
    model_name='deepseek-reasoner',
)

deepseek_chat_flight_ai = DeepSeekFlightAI(
    api_key=deepseek_api_key,
    model_name='deepseek-chat',
)

# --- Define Your Agents for Flight Planning ---

flight_planner = Agent(
    role='Expert Flight Plan Creator',
    goal='Develop optimized and safe flight plans considering all aviation regulations, weather, and aircraft performance, with a strong emphasis on fuel efficiency.',
    backstory="""You are a highly experienced flight operations specialist, adept at creating precise and efficient flight plans.
                 You meticulously consider airspace, weather, NOTAMs, and aircraft limitations to ensure safe and compliant flights. Your expertise includes optimizing flight profiles for maximum fuel efficiency.""",
    tools=[deepseek_reasoner_flight_ai, aviation_database_search_tool, air_traffic_control_tool],
    verbose=False
)

weather_forecaster = Agent(
    role='Aviation Weather Analyst',
    goal='Provide detailed and accurate aviation weather forecasts, identifying potential hazards along the flight route and advising on optimal wind components for fuel efficiency.',
    backstory="""You are a meteorology expert specializing in aviation weather. You analyze METARs, TAFs, and prognostic charts
                 to give pilots the most critical weather information for their flight decisions, with a focus on understanding and leveraging atmospheric conditions for fuel savings.""",
    tools=[aviation_database_search_tool, deepseek_reasoner_flight_ai],
    verbose=False
)

airspace_regulator = Agent(
    role='Airspace and Regulatory Compliance Expert',
    goal='Ensure all flight plans comply with current airspace regulations, NOTAMs, and international aviation laws, while identifying opportunities for direct routing or advantageous track usage.',
    backstory="""You possess an encyclopedic knowledge of aviation regulations and dynamic airspace restrictions.
                 Your role is to flag any potential non-compliance and suggest alternative routings or procedures, always looking for paths that can contribute to fuel efficiency.""",
    tools=[aviation_database_search_tool, deepseek_reasoner_flight_ai],
    verbose=False
)

aircraft_performance_analyst = Agent(
    role='Aircraft Performance Specialist',
    goal='Calculate and advise on optimal aircraft performance parameters for various flight phases, including step climbs, optimal Mach/airspeed, and altitude selection for maximum fuel efficiency, considering payload and wind data.',
    backstory="""You are a master of aircraft systems and performance charts. You can determine the most efficient
                 settings for climb, cruise, and descent, optimizing for fuel burn and time, directly contributing to cost savings and environmental efficiency.""",
    tools=[deepseek_reasoner_flight_ai],
    verbose=False
)

contingency_planner = Agent(
    role='Emergency and Contingency Flight Planner',
    goal='Develop robust contingency plans for unforeseen events during a flight, including alternate airports, emergency procedures, and communication protocols, while also considering fuel-efficient diversion strategies.',
    backstory="""You are a proactive risk assessor, always preparing for the unexpected. You devise alternative strategies
                 and emergency protocols to ensure flight safety even when things go awry, and can also advise on fuel-efficient diversion options.""",
    tools=[aviation_database_search_tool, air_traffic_control_tool, deepseek_reasoner_flight_ai],
    verbose=False
)

# Agents for Payload Calculation
maintenance_coordinator = Agent(
    role='Aircraft Maintenance and Readiness Coordinator',
    goal='Assess aircraft maintenance status, identify any limitations, and ensure airworthiness for the planned flight, including any impacts on performance or weight.',
    backstory="""You are a meticulous aviation maintenance professional. You review maintenance records and schedules
                 to confirm the aircraft is fully airworthy and any maintenance-related weight or performance limitations are accounted for.""",
    tools=[aviation_database_search_tool, deepseek_reasoner_flight_ai],
    verbose=False
)

passenger_coordinator = Agent(
    role='Passenger and Cargo Load Estimator',
    goal='Estimate the total weight of passengers, baggage, and cargo to determine the flight payload, which is crucial for performance and fuel calculations.',
    backstory="""You are an expert in airline operations and logistics, able to accurately forecast passenger and cargo loads.
                 You translate passenger counts and typical baggage allowances into precise weight estimates for flight planning.""",
    tools=[aviation_database_search_tool, deepseek_reasoner_flight_ai],
    verbose=False
)

# --- Define Your Tasks for Flight Planning ---

generate_flight_plan_task = Task(
    description=(
        """Create a detailed flight plan for a flight from KORD (O'Hare International, Chicago) to EDDF (Frankfurt Airport, Germany).\n"""
        """- Aircraft Type: Boeing 777-300ER\n"""
        """- Departure Time: 20:00Z on June 7, 2025\n"""
        """- Include optimal cruising altitude, estimated time en route, and **optimized fuel requirements for maximum efficiency** for a transatlantic crossing.\n"""
        """- Account for current weather conditions, transatlantic winds (jet stream), and any relevant NOTAMs along the route.\n"""
        """- Propose a primary route utilizing the **most fuel-efficient North Atlantic Track (NAT)** for eastbound traffic on June 7, 2025, considering wind components and aircraft performance.\n"""
        """Specifically identify the chosen NAT (e.g., NAT C), its full set of oceanic waypoints (e.g., 48N/010W, 48N/020W, etc.), and its oceanic entry and exit points. """
        """Also include recommended step climbs or optimal Mach/airspeed for fuel efficiency at different stages of the flight. """
        """Include at least two alternate airports in case of diversion (e.g., in Canada, Iceland, UK, or Germany).\n"""
        """The output should be a markdown formatted document outlining the complete care pathway, emphasizing fuel efficiency throughout."""
    ),
    agent=flight_planner,
    output_file='flight_plan_kordtoeddf_b777_fuel_efficient.md', # Updated output file name
    expected_output="""A markdown formatted, comprehensive flight plan including specific NAT details with waypoints, route, altitudes, times, fuel, transatlantic weather, NOTAMs, alternate airports, and clear recommendations for fuel-efficient operations (e.g., optimal Mach, step climbs)."""
)

get_weather_briefing_task = Task(
    description="Retrieve and summarize the en-route weather forecast and significant meteorological phenomena for the flight from KORD to EDDF on June 7, 2025. **Explicitly analyze wind components (jet stream) to recommend the most fuel-efficient North Atlantic Track and optimal cruising altitudes.**",
    agent=weather_forecaster,
    expected_output="""A summary of en-route weather, including transatlantic winds aloft, turbulence, icing, and thunderstorms, with clear recommendations on how to leverage wind for fuel efficiency."""
)

check_airspace_compliance_task = Task(
    description="Identify and list all airspace restrictions, NOTAMs, and special use airspaces that impact the flight route from KORD to EDDF, including oceanic airspace procedures (e.g., NATs). **Ensure the chosen route and any fuel-efficient maneuvers comply with all regulations.**",
    agent=airspace_regulator,
    expected_output="""A list of relevant airspace restrictions, NATs, and NOTAMs with suggested compliance measures for a transatlantic flight, verifying compatibility with fuel efficiency strategies."""
)

# Task for Payload Calculation (Define only once)
# This task needs the outputs from tasks assigned to maintenance_coordinator and passenger_coordinator.
# In a sequential process, the *results* of prior tasks are implicitly available to agents in subsequent tasks
# within the `context`. You just need to list the task objects that should run *before* this one.
calculate_payload_task = Task(
    description=(
        """Calculate the total estimated payload for the Boeing 777-300ER flight from KORD to EDDF on June 7, 2025. """
        """This calculation should consider: """
        """- Number of adult and child passengers and their estimated average weights (Use information gathered by the Passenger Coordinator's tasks). """
        """- Estimated total baggage weight (Use information gathered by the Passenger Coordinator's tasks). """
        """- Any maintenance-related weight limitations or considerations (Use information gathered by the Maintenance Coordinator's tasks). """
        """Provide the total estimated payload in pounds (lbs) or kilograms (kg) and any factors influencing it. """
        """This precise payload is crucial for accurate fuel efficiency calculations."""
    ),
    agent=aircraft_performance_analyst, # Aircraft Performance Analyst is best suited to combine these weights into a total payload
    context=[
        # List the tasks that need to complete *before* this task,
        # whose results are needed as context.
        # You need tasks assigned to maintenance_coordinator and passenger_coordinator to run first.
        # Since these agents don't have specific tasks defined yet, you need to define them.
        # Let's assume you'll define tasks like:
        # get_maintenance_info_task = Task(...) assigned to maintenance_coordinator
        # get_passenger_info_task = Task(...) assigned to passenger_coordinator
        #
        # For this example, we will define simple tasks for these agents
        # to generate the necessary context.

        # Placeholder tasks that the agents will execute to provide the necessary info
        # These tasks need to be defined *before* this task and added to the crew's task list.
        # We will define these below the payload task for clarity, but they must
        # appear earlier in the crew's `tasks` list for a sequential process.
    ],
    output_file='flight_payload_kordtoeddf_b777.md',
    expected_output="""A numerical value for the total estimated payload (passengers + baggage + any maintenance impact) in lbs/kg, with a breakdown."""
)

# Define the tasks that the Maintenance Coordinator and Passenger Coordinator will perform
# Their outputs will serve as context for the calculate_payload_task
get_maintenance_info_task = Task(
    description="Gather maintenance records and identify any relevant limitations or weight considerations for the Boeing 777-300ER (Tail N777UA) for the flight on June 7, 2025.",
    agent=maintenance_coordinator,
    expected_output="""A summary of maintenance status and any weight/performance limitations."""
)

get_passenger_info_task = Task(
    description="Estimate the number of passengers and baggage weight for the KORD-EDDF flight on June 7, 2025.",
    agent=passenger_coordinator,
    expected_output="""Estimated number of adult and child passengers and total baggage weight."""
)


# Now, define calculate_payload_task *again*, including the new tasks in its context.
# This is the corrected definition that replaces the previous placeholder.
calculate_payload_task = Task(
    description=(
        """Calculate the total estimated payload for the Boeing 777-300ER flight from KORD to EDDF on June 7, 2025. """
        """This calculation should consider: """
        """- Number of adult and child passengers and their estimated average weights (Use information gathered by the Passenger Coordinator's tasks, provided in context). """
        """- Estimated total baggage weight (Use information gathered by the Passenger Coordinator's tasks, provided in context). """
        """- Any maintenance-related weight limitations or considerations (Use information gathered by the Maintenance Coordinator's tasks, provided in context). """
        """Provide the total estimated payload in pounds (lbs) or kilograms (kg) and any factors influencing it. """
        """This precise payload is crucial for accurate fuel efficiency calculations."""
    ),
    agent=aircraft_performance_analyst, # Aircraft Performance Analyst is best suited to combine these weights into a total payload
    context=[
        # Provide the *task objects* themselves as context.
        get_maintenance_info_task,
        get_passenger_info_task
    ],
    output_file='flight_payload_kordtoeddf_b777.md',
    expected_output="""A numerical value for the total estimated payload (passengers + baggage + any maintenance impact) in lbs/kg, with a breakdown."""
)


calculate_performance_task = Task(
    description="Based on a Boeing 777-300ER flying KORD to EDDF, with the determined payload, **calculate the optimal cruising altitude(s) for maximum fuel efficiency, considering the latest wind data (jet stream). Recommend specific Mach numbers or airspeeds for different flight segments (e.g., climb, cruise, descent) to minimize fuel burn.** Also provide the estimated fuel burn for the transatlantic primary route.",
    agent=aircraft_performance_analyst,
    context=[get_weather_briefing_task, calculate_payload_task], # Ensure weather and payload context
    expected_output="""Optimal cruising altitude(s), recommended Mach/airspeed for fuel efficiency, and estimated total fuel burn for the specified transatlantic flight."""
)

develop_contingency_task = Task(
    description="Develop a robust contingency plan for the KORD to EDDF flight, including at least three suitable alternate airports (e.g., Gander, Reykjavik, Shannon, Amsterdam) and basic diversion procedures for a transatlantic crossing. **Include considerations for fuel-efficient diversion profiles if an alternate landing is required.**",
    agent=contingency_planner,
    expected_output="""A detailed contingency plan with transatlantic alternate airports and diversion procedures, including fuel-efficient diversion strategies."""
)

# --- Create the Crew for Flight Planning ---

crew = Crew(
    agents=[
        flight_planner,
        weather_forecaster,
        airspace_regulator,
        aircraft_performance_analyst,
        contingency_planner,
        maintenance_coordinator,
        passenger_coordinator
    ],
    tasks=[
        # Order matters for sequential process as tasks depend on each other's output
        get_weather_briefing_task,      # 1. Get weather, including wind for NAT selection
        check_airspace_compliance_task, # 2. Check airspace & potential direct routings
        get_maintenance_info_task,      # 3. Get maintenance info
        get_passenger_info_task,        # 4. Get passenger info
        calculate_payload_task,         # 5. Calculate payload (needs results from 3 and 4)
        calculate_performance_task,     # 6. Calculate optimal performance (alt, Mach, fuel) based on weather (1) and payload (5)
        develop_contingency_task,       # 7. Develop contingency (can now consider fuel-efficient diversions)
        generate_flight_plan_task       # 8. Generate final plan, incorporating all efficiencies
    ],
    verbose=False, # Set to True for detailed execution logs
    process=Process.sequential # Tasks are executed in the order they are defined
)

print("Starting the Flight Planning Crew (AI Agent for Flight Planning) with a focus on fuel efficiency...")
result = crew.kickoff()

print("\n\n########################")
print("## Final Fuel-Efficient Flight Plan Output ##")
print("########################\n")
print(result)

Starting the Flight Planning Crew (AI Agent for Flight Planning) with a focus on fuel efficiency...


########################
## Final Fuel-Efficient Flight Plan Output ##
########################

### Comprehensive Flight Plan: KORD (Chicago O'Hare) to EDDF (Frankfurt Airport)  
**Aircraft:** Boeing 777-300ER (ICAO: B77W)  
**Date:** June 7, 2025  
**Regulations:** FAA Part 91, ICAO Annex 6, EASA Standards  

---

### **Route Summary**  
**Primary Route:**  
`KORD → JOT → YWG → YQR → YAY → NAT Track BRAVO (BETSO → NEEKO → SOMAX → LIMRI) → CRN → NOR → TNT → EDDF`  
- **NAT Track BRAVO** (Optimized for June 7, 2025, based on historical jet stream patterns):  
  `BETSO (61°N/045°W) → NEEKO (60°N/040°W) → SOMAX (58°N/030°W) → LIMRI (56°N/020°W)`  

**Total Distance:** 4,100 NM (Great Circle: 3,950 NM; Route Adherence: +150 NM)  
**Flight Time:** 8 hours 15 minutes (en route)  

---

### **Cruising Altitude & Step Climbs**  
| **Phase**       | **Altitude (FL)** | **Location**            

In [10]:
# You can also read the generated files
try:
    with open('flight_plan_kordtoeddf_b777_fuel_efficient.md', 'r') as f:
        print("\n--- Flight Plan Content ---")
        print(f.read())
except FileNotFoundError:
    print("\nFlight plan file not generated yet or path is incorrect.")

try:
    with open('flight_payload_kordtoeddf_b777.md', 'r') as f:
        print("\n--- Payload Calculation Content ---")
        print(f.read())
except FileNotFoundError:
    print("\nPayload calculation file not generated yet or path is incorrect.")


--- Flight Plan Content ---
### Comprehensive Flight Plan: KORD (Chicago O'Hare) to EDDF (Frankfurt Airport)  
**Aircraft:** Boeing 777-300ER (ICAO: B77W)  
**Date:** June 7, 2025  
**Regulations:** FAA Part 91, ICAO Annex 6, EASA Standards  

---

### **Route Summary**  
**Primary Route:**  
`KORD → JOT → YWG → YQR → YAY → NAT Track BRAVO (BETSO → NEEKO → SOMAX → LIMRI) → CRN → NOR → TNT → EDDF`  
- **NAT Track BRAVO** (Optimized for June 7, 2025, based on historical jet stream patterns):  
  `BETSO (61°N/045°W) → NEEKO (60°N/040°W) → SOMAX (58°N/030°W) → LIMRI (56°N/020°W)`  

**Total Distance:** 4,100 NM (Great Circle: 3,950 NM; Route Adherence: +150 NM)  
**Flight Time:** 8 hours 15 minutes (en route)  

---

### **Cruising Altitude & Step Climbs**  
| **Phase**       | **Altitude (FL)** | **Location**             | **Weight Reduction** |  
|------------------|-------------------|--------------------------|----------------------|  
| Initial Cruise  | FL330             | YWG (Winn